<a href="https://colab.research.google.com/github/alexisdr/uned-tfg-deteccion-eas/blob/main/UNED-TFG-3-train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento del modelo

En este cuaderno se realiza el entrenamiento del modelo

# Parámetros

* ruta_base: ruta en la que se encuentran los datos del corpus
* ruta_dataset: ruta en el que se almacenará el dataset
* modelo_base: modelo que se tomará como punto de partida para realizar el aprendizaje por transferencia.
* num_epochs: númeero de epochs de entrenamiento
* tamanyo_lote: tamaño del lote de trabajo
* nombre_metrica_principal: metrica usada para evaluar el modelo
* token_hugging_face: token usado para almacenar/recuperar datos del área privada de hugging face
* tasa_aprendizaje: tasa de aprendizaje del modelo
* decaimiento_peso: weight decay
* nombre_modelo: nombre del modelo tanto en local como en hugging face
* subir_a_hugging_faces: indica si se debe subir el modelo a hugging face
* privado_en_hugging_face: indica si el modelo debe ser privado en hugging face
* reducir_tamanyo_dataset_para_pruebas: en el caso de realizar pruebas en el entrenamiento o en métricas, permite trabajar con un conjunto de datos reducido.
* procesar_solo_mas_frecuentes: selecciona del dataset el subconjunto de más frecuentes


In [ ]:
ruta_base = '/drive/My Drive/CorpusPFG/'

#Dataset procesado
ruta_dataset = ruta_base + 'Dataset-completo'

#Parametros del modelo
#modelo_base = "allenai/led-base-16384"
#modelo_base = "allenai/longformer-base-4096"
#modelo_base = "bert-base-multilingual-cased"
#modelo_base = "PlanTL-GOB-ES/bsc-bio-es"
modelo_base = "PlanTL-GOB-ES/bsc-bio-ehr-es"
#modelo_base = "PlanTL-GOB-ES/longformer-base-4096-biomedical-clinical-es"
#modelo_base = "alexisdr/uned-tfg-08.91_mas_frecuentes"
num_epochs = 5
tamanyo_lote = 50
nombre_metrica_principal = "metrica_s"
token_hugging_face = "" #INTRODUCIR UN TOKEN DE HUGGING FACE AQUÍ
tasa_aprendizaje = 0.00004
decaimiento_peso = 0.01
nombre_modelo = "uned-tfg-08.99_mas_frecuentes"

subir_a_hugging_faces = True
privado_en_hugging_face = False
reducir_tamanyo_dataset_para_pruebas = False
procesar_solo_mas_frecuentes = True

Instalación de depndencias necesarias

In [ ]:
#Debido a la actualización de dependencias base de Google Colab, 
#es necesario ejecutar este paso, reiniciar el entorno (RESTART RUNTIME), 
#y ejectuar todos los pasos
!pip install transformers[sentencepiece]==4.29.2 evaluate accelerate torch_optimizer datasets
#transformers[sentencepiece] 4.29.2 depends on protobuf<=3.20.2
#tensorflow 2.12.0 requires protobuf>=3.20.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Carga de datos del dataset

In [ ]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [ ]:
from datasets import DatasetDict

dataset = DatasetDict.load_from_disk(ruta_dataset)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 13051
    })
    validation: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 1451
    })
    test: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 3629
    })
    trainMasFrecuentes: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 639
    })
    validationMasFrecuentes: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 72
    })
    testMasFrecuentes: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
     

Permite la selección entre todos los datos disponibles o solo los códigos EAs más frecuentes

In [ ]:
train = "train"
validation = "validation"
test = "test"

if (procesar_solo_mas_frecuentes):
  train = "trainMasFrecuentes"
  validation = "validationMasFrecuentes"
  test = "testMasFrecuentes"


if (reducir_tamanyo_dataset_para_pruebas):  
  print("Antes de la reducción de tamaño")
  print(dataset[train])
  print(dataset[validation])

  dataset_train_reducido = dataset[train].train_test_split(test_size=0.1)
  dataset[train]=dataset_train_reducido["test"]

  dataset_validation_reducido = dataset[train].train_test_split(test_size=0.1)
  dataset[validation]=dataset_validation_reducido["test"]
  
  print("Después de la reducción de tamaño")
  print(dataset[train])  
  print(dataset[validation])

Muestra de datos del dataset de entrenamiento y de las features

In [ ]:
dataset[train][0]

{'acto': 25630686,
 'label': 4,
 'label_str': 'P01.1',
 'label_list': [4],
 'label_list_str': ['P01.1'],
 'informes': ['25630686-165843980.txt'],
 'text': 'hombre juicio clinico recien nacido a termino de peso adecuado para su edad gestacional riesgo infeccioso no confirmado cardiopatia congenita comunicacion interventricular perimembranosa grande sin repercusion comunicacion interauricular tipo ostium secundum moderada con repercusion leve estenosis pulmonar periferica bilateral leve tratamiento captoprilo solucion mg ml ml cada horas\n',
 'json': '[{"apartado": "sexo", "texto": "hombre"}, {"apartado": "juicio clinico", "texto": " juicio clinico recien nacido a termino de peso adecuado para su edad gestacional riesgo infeccioso no confirmado cardiopatia congenita comunicacion interventricular perimembranosa grande sin repercusion comunicacion interauricular tipo ostium secundum moderada con repercusion leve estenosis pulmonar periferica bilateral leve tratamiento captoprilo solucion m

In [ ]:
dataset[train].features

{'acto': Value(dtype='int64', id=None),
 'label': ClassLabel(names=['T38.0X5A', 'T45.515A', 'T50.2X5A', 'Y95', 'P01.1', 'T81.4XXA', 'T45.1X5A', 'Y83.1'], id=None),
 'label_str': Value(dtype='string', id=None),
 'label_list': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'label_list_str': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'informes': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'text': Value(dtype='string', id=None),
 'json': Value(dtype='string', id=None)}

Se crean dos diccionarios para mapear de labels a intero y viceversa

In [ ]:
from datasets import ClassLabel

class2label = dataset[train].features["label"]
id2label = {idx:label for idx, label in enumerate(class2label._int2str)}
label2id = class2label._str2int

print(class2label)
print(id2label)
print(label2id)

ClassLabel(names=['T38.0X5A', 'T45.515A', 'T50.2X5A', 'Y95', 'P01.1', 'T81.4XXA', 'T45.1X5A', 'Y83.1'], id=None)
{0: 'T38.0X5A', 1: 'T45.515A', 2: 'T50.2X5A', 3: 'Y95', 4: 'P01.1', 5: 'T81.4XXA', 6: 'T45.1X5A', 7: 'Y83.1'}
{'T38.0X5A': 0, 'T45.515A': 1, 'T50.2X5A': 2, 'Y95': 3, 'P01.1': 4, 'T81.4XXA': 5, 'T45.1X5A': 6, 'Y83.1': 7}


In [ ]:
class2label.int2str(0)

'T38.0X5A'

# Preprocesado de los datos

El modelo no puede recibir texto como entrada, se ha de convertir a tokens.

In [ ]:
from transformers import AutoTokenizer, RobertaTokenizer
import numpy as np

tokenizer = RobertaTokenizer.from_pretrained(modelo_base, use_auth_token=token_hugging_face)

def preprocesado_datos(dato):
  # toma el texto
  text = dato["text"]
  # lo codifica con tokenizador
  encoding = tokenizer(text, padding="max_length", truncation=True)  
  
  labels = dato["label_list"]
  # crea una matriz del tamaño del texto y las clases a entrenar
  labels_matrix = np.zeros((len(text), class2label.num_classes))

  # asigan a 1 en las posiciones en las que está la clase
  for clase in labels:
    labels_matrix[:, clase] = 1

  # asigna el vector con los clases correspondientes activas
  encoding["labels"] = labels_matrix.tolist()

  return encoding

In [ ]:
dataset[train][0]

{'acto': 25630686,
 'label': 4,
 'label_str': 'P01.1',
 'label_list': [4],
 'label_list_str': ['P01.1'],
 'informes': ['25630686-165843980.txt'],
 'text': 'hombre juicio clinico recien nacido a termino de peso adecuado para su edad gestacional riesgo infeccioso no confirmado cardiopatia congenita comunicacion interventricular perimembranosa grande sin repercusion comunicacion interauricular tipo ostium secundum moderada con repercusion leve estenosis pulmonar periferica bilateral leve tratamiento captoprilo solucion mg ml ml cada horas\n',
 'json': '[{"apartado": "sexo", "texto": "hombre"}, {"apartado": "juicio clinico", "texto": " juicio clinico recien nacido a termino de peso adecuado para su edad gestacional riesgo infeccioso no confirmado cardiopatia congenita comunicacion interventricular perimembranosa grande sin repercusion comunicacion interauricular tipo ostium secundum moderada con repercusion leve estenosis pulmonar periferica bilateral leve tratamiento captoprilo solucion m

In [ ]:
preprocesado_datos(dataset[train][0])

{'input_ids': [0, 3305, 7037, 34783, 44924, 6593, 269, 27403, 262, 1774, 3482, 358, 335, 1149, 12095, 896, 17699, 373, 11098, 1833, 29475, 300, 769, 1018, 3502, 893, 32724, 379, 7233, 45281, 1013, 4447, 596, 4763, 4354, 790, 3502, 893, 41158, 964, 50249, 2281, 428, 6288, 300, 4763, 4354, 790, 5421, 8655, 3232, 25451, 391, 9935, 5421, 653, 2025, 8289, 39580, 13470, 1888, 5479, 5479, 827, 1710, 202, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Procesado de los dataset completos de entrenamiento y validación

In [ ]:
encoded_train_dataset = dataset[train].map(
    preprocesado_datos, batched=True, remove_columns=dataset[train].column_names)
encoded_train_dataset.set_format("torch")

encoded_validation_dataset = dataset[validation].map(
    preprocesado_datos, batched=True, remove_columns=dataset[train].column_names)
encoded_validation_dataset.set_format("torch")

encoded_dataset = DatasetDict(
    {
        train: encoded_train_dataset,
        validation: encoded_validation_dataset
    })

Map:   0%|          | 0/639 [00:00<?, ? examples/s]

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

In [ ]:
encoded_dataset

DatasetDict({
    trainMasFrecuentes: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 639
    })
    validationMasFrecuentes: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 72
    })
})

In [ ]:
dataset[train][0]

{'acto': 25630686,
 'label': 4,
 'label_str': 'P01.1',
 'label_list': [4],
 'label_list_str': ['P01.1'],
 'informes': ['25630686-165843980.txt'],
 'text': 'hombre juicio clinico recien nacido a termino de peso adecuado para su edad gestacional riesgo infeccioso no confirmado cardiopatia congenita comunicacion interventricular perimembranosa grande sin repercusion comunicacion interauricular tipo ostium secundum moderada con repercusion leve estenosis pulmonar periferica bilateral leve tratamiento captoprilo solucion mg ml ml cada horas\n',
 'json': '[{"apartado": "sexo", "texto": "hombre"}, {"apartado": "juicio clinico", "texto": " juicio clinico recien nacido a termino de peso adecuado para su edad gestacional riesgo infeccioso no confirmado cardiopatia congenita comunicacion interventricular perimembranosa grande sin repercusion comunicacion interauricular tipo ostium secundum moderada con repercusion leve estenosis pulmonar periferica bilateral leve tratamiento captoprilo solucion m

In [ ]:
example = encoded_dataset[train][0]
tokenizer.decode(example['input_ids'])

'<s> hombre juicio clinico recien nacido a termino de peso adecuado para su edad gestacional riesgo infeccioso no confirmado cardiopatia congenita comunicacion interventricular perimembranosa grande sin repercusion comunicacion interauricular tipo ostium secundum moderada con repercusion leve estenosis pulmonar periferica bilateral leve tratamiento captoprilo solucion mg ml ml cada horas\n</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [ ]:
encoded_validation_dataset[1]

{'input_ids': tensor([    0,   976,  7037, 34783,  6773, 13610,  1818,  6217,  9050,   391,
         20529,   300,  8579,   355,    93,   893,  3676,   596,  3503,   565,
           547,   528,  1156,   893,   262, 15635,   262,  1708,  1774,  6582,
           316,  6628,   653,  3977,   893,  2298, 10790,   262,  2604,   717,
          1783,  2587,  1161,   893,   288,  3645,  2258,   310,   300,   335,
          1161,   893,  3731, 33733, 15635,   269, 45764,  1888,   637,    76,
          7635, 14166, 26606, 19003,  4996,   202,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [ ]:
example['labels']

tensor([1., 1., 1., 1., 1., 1., 1., 1.])

# Cálculo de la métrica S

In [ ]:
#Calcula el prefico comun entre 2 códigos
def calculo_lcs(codigo_i, codigo_j):
  if codigo_i is None or codigo_j is None:
    return ""

  #Se omiten los puntos existenten en los códigos
  codigo_i = codigo_i.replace(".", "")
  codigo_j = codigo_j.replace(".", "")

  #Tomamos el tamaño mínimo
  size = min(len(codigo_i), len(codigo_j)) 

  lcs_i_j = ''
  for i in range(size):
    if codigo_i[i] == codigo_j[i]:
      lcs_i_j = lcs_i_j + codigo_i[i]
    else:
      return lcs_i_j
  
  return lcs_i_j


#Devuelve la longitud de la cadena de caracteres C
#si esta vale al menos 3, y devuelve 0 si dicha longitud es menor que 3. 
#Esto se debe a que los códigos CIE-10 más cortos contienen al menos 3 caracteres.
def calculo_ic(codigo):
  #Se omiten los puntos existenten en los códigos
  codigo = codigo.replace(".", "")

  tamanyo = len(codigo)
  if (tamanyo < 3):
    return 0
  else:
    return tamanyo

#similitud entre 2 códigos CIE-10 𝑖 y 𝑗:
def calculo_c(codigo_i, codigo_j):
  divisor = calculo_ic(codigo_i) + calculo_ic(codigo_j)
  if (divisor == 0):
    return 0
  dividendo = 2 * calculo_ic(calculo_lcs(codigo_i, codigo_j))
  c_i_j = dividendo / divisor
  return round(c_i_j, 6)

def metrica_s(lista_codigos_i, lista_codigos_j): 
  #las listas deben tener valores 
  if (len(lista_codigos_i) == 0 or len(lista_codigos_j) == 0):
    return 0

  #max (Ng, Ns)
  divisor = max(len(lista_codigos_i), len(lista_codigos_j)) 
  if (divisor == 0):
    return 0
  
  max_c_i_j = 0
  for codigo_j in lista_codigos_j:
    max_local_c_i_j = 0
    for codigo_i in lista_codigos_i:
      c_i_j = calculo_c(codigo_i, codigo_j)
      if (c_i_j > max_local_c_i_j):
        max_local_c_i_j = c_i_j
    max_c_i_j += max_local_c_i_j

  s = max_c_i_j / divisor
  return round(s, 6)

def metrica_s_train (y_true, y_pred):     
    y_true_labels = []
    true_labels = [class2label.int2str([idx])for idx, label in enumerate(y_true) if label == 1.0]
    for label in true_labels:
      y_true_labels.append(label[0])

    y_pred_labels = []
    pred_labels = [class2label.int2str([idx])for idx, label in enumerate(y_pred) if label == 1.0]
    for label in pred_labels:
      y_pred_labels.append(label[0])

    return metrica_s(y_true_labels, y_pred_labels)

Test de los métodos relacionados con la métrica S

In [ ]:
def assert_true(esperado, obtenido):
  assert obtenido == esperado, f"esperado {esperado}, obtenido: {obtenido}"

# TEST calculo_lcs
assert_true('', calculo_lcs('', 'T14.405'))
assert_true('', calculo_lcs('T14.405', ''))
assert_true('', calculo_lcs('T14.405', 'A44.995'))
assert_true('T', calculo_lcs('T14.405', 'T44.995'))
assert_true('T4', calculo_lcs('T42.405', 'T44.995'))
assert_true('T44', calculo_lcs('T44.405', 'T44.995'))
assert_true('T449', calculo_lcs('T44.905', 'T44.995'))
assert_true('T4490', calculo_lcs('T44.906', 'T44.905'))
assert_true('T44905', calculo_lcs('T44.905', 'T44.905'))

# TEST calculo_ic
assert_true(0, calculo_ic(''))
assert_true(0, calculo_ic('T'))
assert_true(0, calculo_ic('T4'))
assert_true(3, calculo_ic('T44'))
assert_true(4, calculo_ic('T44.9'))
assert_true(5, calculo_ic('T44.90'))
assert_true(6, calculo_ic('T44.905'))
assert_true(6, calculo_ic('T44905'))

# TEST calculo_c
assert_true(0, calculo_c('A', 'A'))
assert_true(0, calculo_c('AA', 'AA'))
assert_true(1, calculo_c('AAA', 'AAA'))
assert_true(0, calculo_c('A', 'B'))
assert_true(0, calculo_c('AA', 'BB'))
assert_true(0, calculo_c('AAA', 'AAB'))
assert_true(0, calculo_c('B', 'A'))
assert_true(0.5, calculo_c('AAA.000', 'AAA.111'))
assert_true(0.666667, calculo_c('AAA.100', 'AAA.111'))
assert_true(0.833333, calculo_c('AAA.110', 'AAA.111'))
assert_true(1, calculo_c('AAA.111', 'AAA.111'))
assert_true(0.666667, calculo_c('T44.905', 'T44.995'))

# TEST metrica_s
assert_true(0, metrica_s(['A'], []))
assert_true(0, metrica_s([], ['B']))
assert_true(0, metrica_s(['A'], ['B']))
assert_true(0, metrica_s(['AA'], ['BB']))
assert_true(0, metrica_s(['AAA'], ['BBB']))
assert_true(1, metrica_s(['AAA'], ['AAA']))
assert_true(0.5, metrica_s(['AAA.AAA'], ['AAA.BBB']))
assert_true(0.666667, metrica_s(['AAA.AAA'], ['AAA.ABB']))
assert_true(0.833333, metrica_s(['AAA.AAA'], ['AAA.AAB']))
assert_true(1, metrica_s(['AAA.AAA'], ['AAA.AAA']))
assert_true(0.5, metrica_s(['AAA.AAA', 'B'], ['AAA.AAA']))
assert_true(0.5, metrica_s(['AAA.AAA', 'B'], ['AAA.AAA', 'B']))
assert_true(1, metrica_s(['AAA.AAA', 'BBB'], ['AAA.AAA', 'BBB']))
assert_true(0.833333, metrica_s(['AAA.AAA', 'BBB.BBB'], ['AAA.AAA', 'BBB']))
assert_true(0.555556, metrica_s(['T44.995', 'L76', 'N14.2'], ['T44.900', 'L76']))
assert_true(0.611111, metrica_s(['T44.995', 'L76', 'N14.2'], ['T44.990', 'L76']))
assert_true(0.666667, metrica_s(['T44.995', 'L76', 'N14.2'], ['T44.995', 'L76']))
assert_true(0.916667, metrica_s(['T44.995', 'L76', 'N14.2'], ['T44.995', 'L76', 'N14.0']))
assert_true(1, metrica_s(['T44.995', 'L76', 'N14.2'], ['T44.995', 'L76', 'N14.2']))
assert_true(0.958333, metrica_s(['T44.995', 'L76', 'N14.2', 'AAA'], ['T44.990', 'L76', 'N14.2', 'AAA']))
assert_true(1, metrica_s(['T44.995', 'L76', 'N14.2', 'AAA'], ['T44.995', 'L76', 'N14.2', 'AAA']))
assert_true(0.75, metrica_s(['T44.995', 'L76', 'N14.2', 'AAA'], ['T44.995', 'L76', 'N14.2', 'BBB']))
assert_true(0.25, metrica_s(['T44.995', 'L76', 'N14.2', 'AAA'], ['T44.995', 'YYY', 'XXX', 'BBB']))
assert_true(0.208333, metrica_s(['T44.995', 'L76', 'N14.2', 'AAA'], ['T44.990', 'YYY', 'XXX', 'BBB']))
assert_true(0.166667, metrica_s(['T44.995', 'L76', 'N14.2', 'AAA'], ['T44.900', 'YYY', 'XXX', 'BBB']))
assert_true(0.125, metrica_s(['T44.995', 'L76', 'N14.2', 'AAA'], ['T44.000', 'YYY', 'XXX', 'BBB']))

assert_true(0.000000, metrica_s(['T44.995', 'M96.5', 'N14.2'], []))
assert_true(0.000000, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['AAA']))
assert_true(0.000000, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['AAA', 'BBB']))
assert_true(0.000000, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['AAA', 'BBB', 'CCC']))
assert_true(0.000000, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['AAA', 'BBB', 'CCC', 'DDD']))
assert_true(0.222222, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['T44']))
assert_true(0.266667, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['T44.9']))
assert_true(0.303030, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['T44.99']))
assert_true(0.333333, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['T44.995']))
assert_true(0.333333, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['T44.995', 'BBB']))
assert_true(0.333333, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['T44.995', 'BBB', 'CCC']))
assert_true(0.250000, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['T44.995', 'BBB', 'CCC', 'DDD']))
assert_true(0.619048, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['T44.995', 'M96']))
assert_true(0.666667, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['T44.995', 'M96.5', 'CCC']))
assert_true(0.500000, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['T44.995', 'M96.5', 'CCC', 'DDD']))
assert_true(0.952381, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['T44.995', 'M96.5', 'N14']))
assert_true(1.000000, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['T44.995', 'M96.5', 'N14.2']))
assert_true(0.750000, metrica_s(['T44.995', 'M96.5', 'N14.2'], ['T44.995', 'M96.5', 'N14.2', 'DDD']))

# Entrenamiento del modelo

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    encoded_dataset[train], shuffle=True, batch_size=tamanyo_lote, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    encoded_dataset[validation], batch_size=tamanyo_lote, collate_fn=data_collator
)

for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([50, 512]),
 'attention_mask': torch.Size([50, 512]),
 'labels': torch.Size([50, 8])}

In [ ]:
from tqdm.auto import tqdm
from accelerate import Accelerator
import torch_optimizer as optim
from transformers import AdamW, AutoModelForSequenceClassification, RobertaForSequenceClassification, get_scheduler
from transformers import EvalPrediction
from transformers import TrainingArguments, Trainer
from transformers import get_scheduler, get_cosine_schedule_with_warmup
import torch
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score

codigo_NONE = 'NONE'

def calcular_predicciones(probs, threshold):
  predictions = np.zeros(probs.shape)
  predictions[np.where(probs > threshold)] = 1
  predictions = predictions[0]
  return predictions

#Solo asignar NONE si su probabilidad es mayor que la del resto de códigos y 
#en caso contrario asignar sólo los códigos con mayor probabilidad que NONE
def calcular_predicciones_NONE_mas_probable(probs, threshold):
  predictions = calcular_predicciones(probs, threshold)  
  
  try:
    probabilidad_NONE = probs[0][class2label.str2int(codigo_NONE)]
    #Se ha encontrado NONE entre las predicciones
    if (probabilidad_NONE > 0):
      predictions_mayor_que_prob_NONE = np.zeros(probs.shape)
      predictions_mayor_que_prob_NONE[np.where(probs > probabilidad_NONE)] = 1
      predictions_mayor_que_prob_NONE = predictions_mayor_que_prob_NONE[0]
      # Si hay labels con mejor probabilidad que none nos la quedamos
      if (1 in predictions_mayor_que_prob_NONE):
        predictions = predictions_mayor_que_prob_NONE
      else: #Si no las hay nos quedamos solo con NONE
        predictions[:] = 0
        predictions[class2label.str2int(codigo_NONE)] = 1
  except ValueError:
    pass

  return predictions

#Solo asignar NONE si existe y tiene una probabilidad que supere el umbral
def calcular_predicciones_existe_NONE(probs, threshold):
  predictions = calcular_predicciones(probs, threshold)  

  try:
    probabilidad_NONE = probs[0][class2label.str2int(codigo_NONE)]
    #Se ha encontrado NONE entre las predicciones, nos la quedamos de forma exclusiva
    if (probabilidad_NONE > threshold):
        predictions[:] = 0
        predictions[class2label.str2int(codigo_NONE)] = 1
  except ValueError:
    pass

  return predictions

def calculo_metricas_multi_label(predictions, labels, threshold=0.9):
    # aplica el sigmoid a las predicciones
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # a partir del umbral se marcan las predicciones
    y_pred = calcular_predicciones_NONE_mas_probable(probs, threshold)
    y_true = labels[0]
    
    metric_average = "micro"
    precision_score_value = precision_score(y_true, y_pred, average=metric_average)
    recall_score_value = recall_score(y_true, y_pred, average=metric_average)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average=metric_average)
    #roc_auc = roc_auc_score(y_true, y_pred, average=metric_average)
    accuracy = accuracy_score(y_true, y_pred)
    metrica_s_value = metrica_s_train(y_true, y_pred)
    
    metrics = {
      'precision_score': precision_score_value,
      'recall_score': recall_score_value,
      'f1': f1_micro_average,
      #'roc_auc': roc_auc,
      'accuracy': accuracy, 
      'metrica_s': metrica_s_value}
    return metrics

def calculo_metricas(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions

    return calculo_metricas_multi_label(predictions=preds, labels=p.label_ids)

def entrenamiento_modelo():

  model = RobertaForSequenceClassification.from_pretrained(
        modelo_base, 
        num_labels=class2label.num_classes, 
        id2label = id2label, 
        label2id = label2id,
        problem_type = "multi_label_classification",
        use_auth_token = token_hugging_face) 

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)

  accelerator = Accelerator()
  optimizer = torch.optim.AdamW(model.parameters(), lr=tasa_aprendizaje)
  train_dl, eval_dl, model, optimizer = accelerator.prepare(
      train_dataloader, eval_dataloader, model, optimizer
  )

  num_training_steps = num_epochs * len(train_dataloader)
  num_warmup_steps = 2 * len(train_dataloader)
  lr_scheduler = get_cosine_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=num_warmup_steps,
      num_training_steps=num_training_steps,
  )

  progress_bar = tqdm(range(num_training_steps))

  training_args = TrainingArguments(
      optim="adamw_torch",
      evaluation_strategy = "epoch", 
      save_strategy = "epoch",
      load_best_model_at_end=True,
      output_dir= nombre_modelo, 
      per_device_train_batch_size=tamanyo_lote,
      per_device_eval_batch_size=tamanyo_lote,
      num_train_epochs=num_epochs,
      learning_rate=tasa_aprendizaje,
      weight_decay=decaimiento_peso,
      metric_for_best_model=nombre_metrica_principal,
      hub_token=token_hugging_face,
      hub_private_repo=privado_en_hugging_face,
      push_to_hub=subir_a_hugging_faces) 

  trainer = Trainer(
      model=model,
      args=training_args,
      data_collator=data_collator,
      train_dataset=encoded_dataset[train],
      eval_dataset=encoded_dataset[validation],
      compute_metrics=calculo_metricas,
      optimizers=(optimizer,lr_scheduler)
  )

  trainer.train()

  if (subir_a_hugging_faces):
    tokenizer.push_to_hub(
        nombre_modelo, 
        private=privado_en_hugging_face, 
        use_auth_token=token_hugging_face)

In [ ]:
from accelerate import notebook_launcher
notebook_launcher(entrenamiento_modelo)

Launching training on one GPU.


Some weights of the model checkpoint at PlanTL-GOB-ES/bsc-bio-ehr-es were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/bsc-bio-ehr-es and are newly initialized: ['classifier.out_proj.bias', 'classifier.d

  0%|          | 0/65 [00:00<?, ?it/s]

Cloning https://huggingface.co/alexisdr/uned-tfg-08.99_mas_frecuentes into local empty directory.


Epoch,Training Loss,Validation Loss,Precision Score,Recall Score,F1,Accuracy,Metrica S
1,No log,0.419725,0.000000,0.000000,0.000000,0.000000,0
2,No log,0.190565,0.000000,0.000000,0.000000,0.000000,0
3,No log,0.102208,0.625000,0.625000,0.625000,0.625000,0.625000
4,No log,0.076501,1.000000,1.000000,1.000000,1.000000,1.000000
5,No log,0.072931,1.000000,1.000000,1.000000,1.000000,1.000000
